# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [3]:
import numpy as np
from sklearn import datasets, model_selection, tree, metrics

In [4]:
boston = datasets.load_boston()
X = boston.data
y=boston.target
X.shape #(506, 13)
print(X[:1])
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, shuffle=False)
X_train.shape #(379, 13)
print(X_train[:1])
print()
print(X_test[:1])
l = int(len(X)*.75)
print(X[l:l+1])

[[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
  4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]]
[[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
  4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00]]

[[ 17.8667   0.      18.1      0.       0.671    6.223  100.       1.3861
   24.     666.      20.2    393.74    21.78  ]]
[[ 17.8667   0.      18.1      0.       0.671    6.223  100.       1.3861
   24.     666.      20.2    393.74    21.78  ]]


## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [5]:
def gbm_predict(X, algs, coeffs):
    #return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
    return np.sum( coeff * algo.predict(X) for algo, coeff in zip(algs, coeffs) )
     
def my_gbm( step=0.9 ):      
    algs = []
    coeffs = []

    for i in range(50):
        b = tree.DecisionTreeRegressor( max_depth=5, random_state=42 )
        z = gbm_predict(X_train, algs, coeffs)
        b.fit( X_train, y_train - z ) # на первом цикле (для b0) z=0
        algs.append(b)
        coeffs.append(step)

        mse = metrics.mean_squared_error( y_test, gbm_predict(X_test, algs, coeffs) )
        print( 'MSE:', mse )

    rmse = mse ** 0.5
    print( 'RMSE', rmse ) 
    #5.47685241245506
    #5.476650974168948
    
my_gbm()

MSE: 18.131625135794057
MSE: 21.81967702217562
MSE: 24.032616657775254
MSE: 24.64276158058703
MSE: 27.310220412002263
MSE: 26.597879127684777
MSE: 27.688977556189453
MSE: 28.2001068578865
MSE: 30.220883003231467
MSE: 30.357484200450294
MSE: 30.4192448242861
MSE: 30.346308180368304
MSE: 30.333055279102027
MSE: 30.397353944296945
MSE: 29.649366748778963
MSE: 29.994464219065605
MSE: 29.980294885960696
MSE: 29.853607830324528
MSE: 29.95289288786412
MSE: 29.946874189513593
MSE: 29.99351372240485
MSE: 29.96356288947434
MSE: 29.97151285268748
MSE: 30.009579042891616
MSE: 30.027083849484367
MSE: 30.04089214117322
MSE: 30.027153870603033
MSE: 30.04187553994958
MSE: 30.206499404939184
MSE: 30.00124785569655
MSE: 30.00834347487608
MSE: 30.008793870782025
MSE: 29.990684627071467
MSE: 29.988810193632336
MSE: 29.9868447763648
MSE: 29.99709297172252
MSE: 30.013042373621772
MSE: 30.005525034773076
MSE: 29.98817542720594
MSE: 29.995280234397132
MSE: 29.99418849833665
MSE: 29.990179010009577
MSE: 29.983

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [6]:
def my_gbm2():      
    algs = []
    coeffs = []

    for i in range(50):
        b = tree.DecisionTreeRegressor( max_depth=5, random_state=42 )
        z = gbm_predict(X_train, algs, coeffs)
        b.fit( X_train, y_train - z ) # на первом цикле (для b0) z=0
        algs.append(b)
        coeffs.append( 0.9/(1.0+i) )

        mse = metrics.mean_squared_error( y_test, gbm_predict(X_test, algs, coeffs) )
        print( 'MSE:', mse )

    rmse = mse ** 0.5
    print( 'RMSE', rmse ) 
    #5.47685241245506
    #5.476650974168948
my_gbm2()    

MSE: 18.131625135794057
MSE: 19.462169988184364
MSE: 19.28248964708356
MSE: 19.880407711297753
MSE: 20.23764722065624
MSE: 20.401379543283866
MSE: 20.70704451735892
MSE: 21.237789452263918
MSE: 21.467816143288545
MSE: 21.630951294413123
MSE: 21.701929427327652
MSE: 21.763764287465534
MSE: 21.965801550883807
MSE: 22.129816399667774
MSE: 22.164360486094463
MSE: 22.334789838693236
MSE: 22.44297386817373
MSE: 22.488776787274467
MSE: 22.531684811378856
MSE: 22.594075511045553
MSE: 22.652649602390344
MSE: 22.69053170018508
MSE: 22.69183643989601
MSE: 22.675377398768408
MSE: 22.67549568509912
MSE: 22.667677379698908
MSE: 22.62773935236995
MSE: 22.725941561741745
MSE: 22.75740192481981
MSE: 22.838234556383842
MSE: 22.801990265704084
MSE: 22.869653530016457
MSE: 22.896225045916644
MSE: 22.914490311242385
MSE: 22.902908334477964
MSE: 22.952772656605568
MSE: 22.96280427615932
MSE: 23.019635218155
MSE: 23.063467432342584
MSE: 23.061916742678836
MSE: 23.05454233657498
MSE: 23.063595937809055
MSE: 2

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

In [28]:
# ответы: 1 4
import pandas as pd
from sklearn import cross_validation
from sklearn.ensemble import GradientBoostingRegressor

rmses = []
scoring = []
n_trees = list(range(1, 20))
for n_tree in n_trees:
    est = GradientBoostingRegressor(learning_rate=0.9, max_depth=5, n_estimators=n_tree, random_state=42)
    est.fit(X_train, y_train)
    rmse = metrics.mean_squared_error( y_test, est.predict(X_test)) **0.5    
    rmses.append(rmse)
    print('RMSE:', rmse)    


RMSE: 6.0178253371157
RMSE: 5.6359349825901175
RMSE: 5.821707179663399
RMSE: 5.843890766567132
RMSE: 6.123495314991765
RMSE: 6.057373778741594
RMSE: 6.1561857215105755
RMSE: 6.185768020905074
RMSE: 6.295985841934424
RMSE: 6.295043123835479
RMSE: 6.299017959258736
RMSE: 6.292578508882951
RMSE: 6.291031422489822
RMSE: 6.29667390907419
RMSE: 6.230912864439426
RMSE: 6.267721540901985
RMSE: 6.26621140687479
RMSE: 6.252457243703203
RMSE: 6.261268006117562


In [29]:
rmses = []
scoring = []
n_depths = list(range(1, 20, 2))
for n_depth in n_depths:
    est = GradientBoostingRegressor(learning_rate=0.9, max_depth=n_depth, n_estimators=50, random_state=42)
    est.fit(X_train, y_train)
    rmse = metrics.mean_squared_error( y_test, est.predict(X_test)) **0.5    
    rmses.append(rmse)
    print('RMSE:', rmse)    


RMSE: 7.040180585989451
RMSE: 5.383576139877358
RMSE: 6.280206100045124
RMSE: 6.565563505546183
RMSE: 6.045837871592224
RMSE: 6.632901263160843
RMSE: 6.696203656409291
RMSE: 7.275616306896002
RMSE: 7.28036456297363
RMSE: 6.122602908965554


## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [30]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
rmse = metrics.mean_squared_error(y_test, lr.predict(X_test)) ** 0.5
print('rmse:', rmse)


rmse: 8.270468034938213
